In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, Activation, Dropout
from tensorflow.keras import Input, Model

In [9]:
DATA_DIR = "../../pamap2+physical+activity+monitoring/PAMAP2_Dataset/Protocol"

In [10]:
subjects_list = [
    "subject101.dat",
    "subject102.dat",
    "subject103.dat",
    "subject104.dat",
    "subject105.dat",
    "subject106.dat",
    "subject107.dat",
    "subject108.dat",
    "subject109.dat",
]

In [41]:
filepath = os.path.join(DATA_DIR, subjects_list[1])
df = pd.read_csv(filepath, sep=r"\s+", header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
0,5.64,0,NaN,33.0,2.79143,7.55389,-7.06374,2.87553,7.88823,-6.76139,...,-0.027148,-0.031190,-0.040897,-47.7695,-2.58701,59.8481,-0.012871,0.747947,-0.079841,0.658813
1,5.65,0,NaN,33.0,2.86086,7.43814,-7.21626,2.84248,7.63164,-6.85140,...,0.012804,-0.036384,-0.014845,-47.7624,-2.81438,60.3407,0.014025,-0.748410,0.079043,-0.658360
2,5.66,0,100.0,33.0,2.80997,7.09430,-7.52376,2.79373,7.43545,-7.07744,...,-0.001695,-0.034529,-0.040074,-47.5393,-2.38909,60.9623,0.012714,-0.749508,0.075286,-0.657577
3,5.67,0,NaN,33.0,2.77138,6.98077,-7.52396,2.80627,7.29929,-7.24326,...,0.055437,-0.011560,0.015434,-47.4246,-2.72082,60.7103,0.012639,-0.749201,0.076030,-0.657843
4,5.68,0,NaN,33.0,2.76710,7.13119,-7.67848,2.78766,7.11797,-7.48445,...,0.017184,-0.034663,0.007239,-47.5412,-2.60153,60.3412,0.012571,-0.749291,0.076346,-0.657705


In [42]:
df = df[[0, 1, 4, 5, 6, 10, 11, 12]]
columns = [
    "timestamp",
    "action",
    "acc_x",
    "acc_y",
    "acc_z",
    "gyr_x",
    "gyr_y",
    "gyr_z",
]
df.columns = columns
df = df.dropna(axis=0)
df.head()

,timestamp,action,acc_x,acc_y,acc_z,gyr_x,gyr_y,gyr_z
0,5.64,0,2.79143,7.55389,-7.06374,1.01640,-0.289410,1.38207
1,5.65,0,2.86086,7.43814,-7.21626,1.08269,-0.393965,1.60935
2,5.66,0,2.80997,7.09430,-7.52376,1.14829,-0.521087,1.80748
3,5.67,0,2.77138,6.98077,-7.52396,1.24888,-0.511985,1.95031
4,5.68,0,2.76710,7.13119,-7.67848,1.27178,-0.525704,2.01367


In [43]:
df["action"].value_counts()

action
0     183369
4      31742
17     28869
7      28694
3      25572
6      25107
1      23429
2      22343
16     20681
12     17341
13     15207
24     13074
5       8843
Name: count, dtype: int64

In [26]:
groups = {action: df_group for action, df_group in df.groupby('action')}

In [27]:
def create_windows_from_group(df_group, window_size=120):
    imu_data = df_group.iloc[:, 2:].values  # taking only the 6 IMU columns
    windows = []
    for i in range(len(imu_data) - window_size + 1):
        windows.append(imu_data[i : i + window_size])
    return np.array(windows)

In [28]:
window_size = 120
X_list = []
y_list = []

for action, df_group in groups.items():
    windows = create_windows_from_group(df_group, window_size=window_size)
    X_list.append(windows)

    # Assign labels (one label per window)
    y_list.append(np.array([action] * len(windows)))

X = np.vstack(X_list)
y = np.concatenate(y_list)

In [29]:
print(X.shape)
print(y.shape)

(373416, 120, 6)
(373416,)


In [30]:
def sample_batch(X, y, batch_size=32):
    idx = random.sample(range(len(X)), batch_size)
    return X[idx], y[idx]

In [31]:
batch_x, batch_y = sample_batch(X, y, batch_size=32)
print(batch_x.shape)  # (32, 120, 6)
print(batch_y.shape)  # (32,)

(32, 120, 6)
(32,)


In [32]:
def simple_tcn(input_shape,
               num_layers=3,
               filters=64,
               kernel_size=3,
               num_classes=5,
               dropout_rate=0.0):
    
    inputs = Input(shape=input_shape)
    x = inputs

    for i in range(num_layers):
        x = Conv1D(filters=filters,
                   kernel_size=kernel_size,
                   padding='causal',
                   dilation_rate=2**i)(x)
        x = Activation('relu')(x)
        if dropout_rate > 0:
            x = Dropout(dropout_rate)(x)

    # ✅ Either predict per timestep OR full sequence
    # Option A: Use only last timestep output
    x = x[:, -1, :]  # shape -> (batch, filters)

    # ✅ Final classification layer
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    return model


In [37]:
# Input shape: (timesteps, features)
input_shape = (120, 6)

model = simple_tcn(
    input_shape=input_shape,
    num_layers=2,
    filters=32,
    kernel_size=8,
    dropout_rate=0.1,
    num_classes=25,
)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 120, 6)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 120, 32)        │         1,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 120, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 120, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 120, 32)        │         8,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 120, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 120, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ get_item_2 (GetItem)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 25)             │           825 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,617 (41.47 KB)

 Trainable params: 10,617 (41.47 KB)

 Non-trainable params: 0 (0.00 B)

In [38]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

In [39]:
import numpy as np

print("NaNs in X:", np.isnan(X).any())
print("Infs in X:", np.isinf(X).any())

print("NaNs in y:", np.isnan(y).any())
print("Infs in y:", np.isinf(y).any())


NaNs in X: False
Infs in X: False
NaNs in y: False
Infs in y: False


In [40]:
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=32,
    shuffle=True
)

2025-10-08 16:23:30.797635: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 860348160 exceeds 10% of free system memory.


Epoch 1/10
9336/9336 ━━━━━━━━━━━━━━━━━━━━ 64s 7ms/step - accuracy: 0.7060 - loss: 0.9327 - val_accuracy: 0.7639 - val_loss: 0.7373
Epoch 2/10
9336/9336 ━━━━━━━━━━━━━━━━━━━━ 64s 7ms/step - accuracy: 0.7635 - loss: 0.7490 - val_accuracy: 0.7956 - val_loss: 0.6518
Epoch 3/10
9336/9336 ━━━━━━━━━━━━━━━━━━━━ 62s 7ms/step - accuracy: 0.7766 - loss: 0.7071 - val_accuracy: 0.8109 - val_loss: 0.6176
Epoch 4/10
9336/9336 ━━━━━━━━━━━━━━━━━━━━ 63s 7ms/step - accuracy: 0.7838 - loss: 0.6879 - val_accuracy: 0.8070 - val_loss: 0.6071
Epoch 5/10
9336/9336 ━━━━━━━━━━━━━━━━━━━━ 63s 7ms/step - accuracy: 0.7878 - loss: 0.6754 - val_accuracy: 0.8113 - val_loss: 0.6169
Epoch 6/10
9336/9336 ━━━━━━━━━━━━━━━━━━━━ 65s 7ms/step - accuracy: 0.7915 - loss: 0.6643 - val_accuracy: 0.8051 - val_loss: 0.6065
Epoch 7/10
9336/9336 ━━━━━━━━━━━━━━━━━━━━ 65s 7ms/step - accuracy: 0.7935 - loss: 0.6573 - val_accuracy: 0.8191 - val_loss: 0.5827
Epoch 8/10
9336/9336 ━━━━━━━━━━━━━━━━━━━━ 64s 7ms/step - accuracy: 0.7960 - loss: 0

In [41]:
model.save("tcn_model.h5")

In [42]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("tcn_model.tflite", "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp9vzd1sqo/assets


INFO:tensorflow:Assets written to: /tmp/tmp9vzd1sqo/assets


Saved artifact at '/tmp/tmp9vzd1sqo'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 120, 6), dtype=tf.float32, name='keras_tensor_18')
Output Type:
  TensorSpec(shape=(None, 25), dtype=tf.float32, name=None)
Captures:
  140470775805584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140470775805936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140470775805056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140470776438048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140470776440688: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140470776441392: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1759934324.218029   17909 tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
W0000 00:00:1759934324.218691   17909 tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-10-08 16:38:44.224537: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp9vzd1sqo
2025-10-08 16:38:44.225208: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-10-08 16:38:44.225221: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmp9vzd1sqo
I0000 00:00:1759934324.233950   17909 mlir_graph_optimization_pass.cc:437] MLIR V1 optimization pass is not enabled
2025-10-08 16:38:44.236230: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-10-08 16:38:44.281582: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmp9vzd1sqo
2025-10-08 16:38:44.291926: I tensorflow/cc/saved_model/loader.cc:471] SavedModel 

In [43]:
def representative_data_gen():
    for i in range(100):
        # X_train should be your training samples, shape (N,120,6)
        sample = X_train[i:i+1].astype('float32')
        yield [sample]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

tflite_quant_model = converter.convert()

with open("tcn_model_int8.tflite", "wb") as f:
    f.write(tflite_quant_model)

INFO:tensorflow:Assets written to: /tmp/tmpl6dl5vbb/assets


INFO:tensorflow:Assets written to: /tmp/tmpl6dl5vbb/assets


Saved artifact at '/tmp/tmpl6dl5vbb'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 120, 6), dtype=tf.float32, name='keras_tensor_18')
Output Type:
  TensorSpec(shape=(None, 25), dtype=tf.float32, name=None)
Captures:
  140470775805584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140470775805936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140470775805056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140470776438048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140470776440688: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140470776441392: TensorSpec(shape=(), dtype=tf.resource, name=None)


/home/shenoy/focus-anchor/NiT-Sparkathon-Focus-Anchor/venv/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:863: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
W0000 00:00:1759934349.847061   17909 tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
W0000 00:00:1759934349.847082   17909 tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-10-08 16:39:09.847322: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpl6dl5vbb
2025-10-08 16:39:09.847916: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-10-08 16:39:09.847926: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpl6dl5vbb
2025-10-08 16:39:09.852814: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-10-08 16:39:09.877726: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on 

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, Activation, Dropout
from tensorflow.keras import Input, Model

In [7]:
# Load TFLite model
interpreter = tf.lite.Interpreter(model_path="./sample_trained_model/tcn_model_int8.tflite")
interpreter.allocate_tensors()

# Get input & output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Example inference
sample = np.zeros((1,120,6)).astype("int8")
interpreter.set_tensor(input_details[0]['index'], sample)
interpreter.invoke()
output = interpreter.get_tensor(output_details[0]['index'])
print("Prediction:", output)

Prediction: [[-128  127 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128
  -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128]]


/home/shenoy/focus-anchor/NiT-Sparkathon-Focus-Anchor/venv/lib/python3.10/site-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
